# **Cálculo do Tracking Error em Python**

Sempre que se realiza investimentos no mercado financiero é necessário saber se o seu investimento realmente é bom, por isso devemos compara-lo com outros investimentos do mesmo setor. O nome da metrica que utiliza-se muito no mercado financiero, chama-se _benchmark_. 

O _benchmark_ é um indice composto pela média de retornos de determinada classe de ativos, exemplo o Ibovespa, composto por uma carteira teórica de 350 ações negociadas na Bolsa de Valores, esse indice serve para comparar com a rentabilidade de portfólios de renda váriavel.  

A métrica de _Tracking Error_ é comparada juntamente com o _benchmark_. O _Tracking Error_ é uma métrica que comparar o desvio padrão do retorno de determinado fundo com o benchmark.

A Fórmula do _Tracking Error_

$$TE = \sqrt{\sum{\frac{(F-I)^2}{N-1}}}$$

In [1]:
# Importanto as bibliotecas
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yfin

yfin.pdr_override()

In [2]:
# leitura dos dados
symbols = ["AAPL", "^GSPC"]

data = pdr.DataReader(
    symbols,
    start = '2020-01-01',
    end = '2023-06-30')['Adj Close']

[*********************100%***********************]  2 of 2 completed


In [3]:
data.head()

,AAPL,^GSPC
Date,,
2020-01-02,73.347946,3257.850098
2020-01-03,72.634850,3234.850098
2020-01-06,73.213615,3246.280029
2020-01-07,72.869301,3237.179932
2020-01-08,74.041489,3253.050049


In [4]:
# Calculando o retorno do ativo e do benchmark
data['stock_returns'] = data['AAPL'].pct_change()
data['bench_returns'] = data['^GSPC'].pct_change()

In [5]:
data.head()

,AAPL,^GSPC,stock_returns,bench_returns
Date,,,,
2020-01-02,73.347946,3257.850098,NaN,NaN
2020-01-03,72.634850,3234.850098,-0.009722,-0.007060
2020-01-06,73.213615,3246.280029,0.007968,0.003533
2020-01-07,72.869301,3237.179932,-0.004703,-0.002803
2020-01-08,74.041489,3253.050049,0.016086,0.004902


In [6]:
# Calculo do benchmark
data['TE'] = (data["stock_returns"] - data["bench_returns"])
data.head()

,AAPL,^GSPC,stock_returns,bench_returns,TE
Date,,,,,
2020-01-02,73.347946,3257.850098,NaN,NaN,NaN
2020-01-03,72.634850,3234.850098,-0.009722,-0.007060,-0.002662
2020-01-06,73.213615,3246.280029,0.007968,0.003533,0.004435
2020-01-07,72.869301,3237.179932,-0.004703,-0.002803,-0.001900
2020-01-08,74.041489,3253.050049,0.016086,0.004902,0.011184


In [7]:
te = data['TE'].std() * 100
te

1.2952717976271269

In [8]:
te * np.sqrt(252)

20.56180234054176

In [9]:
(data["stock_returns"] - data["bench_returns"]).std() * 100

1.2952717976271269

In [10]:
(data["stock_returns"] - data["bench_returns"]).std() * 100 * np.sqrt(252)

20.56180234054176

# Construindo uma função de Tracking Error

In [15]:
def tracking_error(stocks, benchmark, start_date, end_date):
    data  = pd.DataFrame()
    bench = pd.DataFrame()
    data  = pdr.DataReader(stocks, 
                           start = start_date, 
                           end = end_date)['Adj Close']
    bench = pdr.DataReader(benchmark, 
                           start = start_date, 
                           end = end_date)['Adj Close']
    
    for i in data:
        data['daily_return_' + i] = data[i].pct_change()
        bench['daily_return_bench'] = bench.pct_change()
        te_i = (data['daily_return_' + i] - bench['daily_return_bench']).std()
        print(i, "Tracking Error is {0:.2f}".format(te_i * 100) + '%')
        print(i, "Annaulized Tracking Error is {0:.2f}".format(te_i * np.sqrt(252) * 100) + '%')

In [16]:
tracking_error(['AAPL', 'AMZN', 'F'], ["^GSPC"], '2020-01-01', '2020-12-31')

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  1 of 1 completed
AAPL Tracking Error is 1.67%
AAPL Annaulized Tracking Error is 26.45%


/var/folders/c7/ddgj700902b0rpl5xc_zwx8m0000gn/T/ipykernel_3177/513182884.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bench['daily_return_bench'] = bench.pct_change()


TypeError: setting an array element with a sequence.